In [3]:
import re
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [4]:
# read csv
path='../data/heb_scrape.csv'
heb_df=pd.read_csv(path)
heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,", 1 gal",($0.05/oz)
1,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,"fat Milk, 1 gal",($0.05/oz)
2,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS. Primo Pick.,671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,", 1/2 gal",($0.07/oz)
3,H-E-B,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,", 1/2 gal",($0.04/oz)
4,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,", 1/2 gal",($0.04/oz)
...,...,...,...,...,...,...,...,...,...,...
255,Hill Country Fare,Dairy & Eggs/Milk,0.0,GO LOCAL.,314131,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Reduced Fat 2% Milk, 1 gal",3.07,", 1 gal",($0.03/oz)
256,Hill Country Fare,Dairy & Eggs/Milk,0.0,GO LOCAL.,314124,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Whole Milk, 1 gal",3.07,", 1 gal",($0.03/oz)
257,Hill Country Fare,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL.,314128,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Fat Free Milk, 1 gal",3.07,", 1 gal",($0.03/oz)
258,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,658779,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Chocolate Milk, 1 gal",3.28,", 1 gal",($0.03/oz)


In [5]:
heb_df.groupby(['uomSalePrice']).agg({'id':'count'})

,id
uomSalePrice,
($0.03/fl oz),1
($0.03/oz),14
($0.04/fl oz),2
($0.04/oz),8
($0.05/fl oz),1
($0.05/oz),61
($0.06/fl oz),2
($0.06/oz),19
($0.07/fl oz),1


In [6]:
# clean size 
clean_heb_df=pd.DataFrame()
clean_heb_df=heb_df
clean_heb_df['size']=heb_df['size'].str.rsplit(pat=',').str[1]
clean_heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,1 gal,($0.05/oz)
1,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,1 gal,($0.05/oz)
2,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS. Primo Pick.,671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,1/2 gal,($0.07/oz)
3,H-E-B,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
4,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
...,...,...,...,...,...,...,...,...,...,...
255,Hill Country Fare,Dairy & Eggs/Milk,0.0,GO LOCAL.,314131,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Reduced Fat 2% Milk, 1 gal",3.07,1 gal,($0.03/oz)
256,Hill Country Fare,Dairy & Eggs/Milk,0.0,GO LOCAL.,314124,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Whole Milk, 1 gal",3.07,1 gal,($0.03/oz)
257,Hill Country Fare,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL.,314128,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Fat Free Milk, 1 gal",3.07,1 gal,($0.03/oz)
258,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,658779,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Chocolate Milk, 1 gal",3.28,1 gal,($0.03/oz)


In [7]:
clean_heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,1 gal,($0.05/oz)
1,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,1 gal,($0.05/oz)
2,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS. Primo Pick.,671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,1/2 gal,($0.07/oz)
3,H-E-B,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
4,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
...,...,...,...,...,...,...,...,...,...,...
255,Hill Country Fare,Dairy & Eggs/Milk,0.0,GO LOCAL.,314131,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Reduced Fat 2% Milk, 1 gal",3.07,1 gal,($0.03/oz)
256,Hill Country Fare,Dairy & Eggs/Milk,0.0,GO LOCAL.,314124,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Whole Milk, 1 gal",3.07,1 gal,($0.03/oz)
257,Hill Country Fare,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL.,314128,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Fat Free Milk, 1 gal",3.07,1 gal,($0.03/oz)
258,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,658779,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Chocolate Milk, 1 gal",3.28,1 gal,($0.03/oz)


In [8]:
# # unit of measure sales price - per oz float 
# clean_heb_df['uomSalePrice']=heb_df['uomSalePrice'].str.strip('($/oz/fl )').astype(float)


In [9]:
# convert features to list
clean_heb_df['features']=heb_df['features'].str.split(pat='.').str[:-1]

In [10]:
clean_heb_df['features'][3]

['  CARB AWARE', '  LOW SATURATED FAT', '  GO LOCAL', '  SELECT INGREDIENTS']

In [11]:
clean_heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,"[ ORGANIC, GO LOCAL]",931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,1 gal,($0.05/oz)
1,Central Market,Dairy & Eggs/Milk,0.0,"[ ORGANIC, GO LOCAL]",931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,1 gal,($0.05/oz)
2,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS, Primo Pick]",671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,1/2 gal,($0.07/oz)
3,H-E-B,Dairy & Eggs/Milk,0.0,"[ CARB AWARE, LOW SATURATED FAT, GO LOCAL...",314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
4,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS]",314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
...,...,...,...,...,...,...,...,...,...,...
255,Hill Country Fare,Dairy & Eggs/Milk,0.0,[ GO LOCAL],314131,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Reduced Fat 2% Milk, 1 gal",3.07,1 gal,($0.03/oz)
256,Hill Country Fare,Dairy & Eggs/Milk,0.0,[ GO LOCAL],314124,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Whole Milk, 1 gal",3.07,1 gal,($0.03/oz)
257,Hill Country Fare,Dairy & Eggs/Milk,0.0,"[ CARB AWARE, LOW SATURATED FAT, GO LOCAL]",314128,https://images.heb.com/is/image/HEBGrocery/prd...,"Hill Country Fare Fat Free Milk, 1 gal",3.07,1 gal,($0.03/oz)
258,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS]",658779,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Chocolate Milk, 1 gal",3.28,1 gal,($0.03/oz)


In [12]:
milk_names=clean_heb_df.groupby(['name']).agg({'id':'count'})
milk_names

,id
name,
"A Taste of Thai Unsweetened Coconut Milk, 13.5 oz",1
"Banana Wave Original Banana Milk, 32 oz",1
"Blue Diamond Almond Breeze Chocolate Almond Milk, 1/2 gal",1
"Blue Diamond Almond Breeze Original Almond Milk, 1/2 gal",1
"Blue Diamond Almond Breeze Original Non-dairy Beverage, 32 oz",1
...,...
"Yoo-hoo Chocolate Drink Family Pack, 32 pk",1
"Yoo-hoo Chocolate Drink, 10 pk",1
"Yoo-hoo Chocolate Drink, 3 pk",1


In [13]:
clean_heb_df['type']=clean_heb_df['name'].str.rsplit(pat=',').str[0]
clean_heb_df['type']

0                     Central Market Organics Whole Milk
1      Central Market Organics Reduced Fat 2% Milkfat...
2      H‑E‑B Select Ingredients MooTopia Lactose Free...
3                 H‑E‑B Select Ingredients Fat Free Milk
4               H‑E‑B Select Ingredients 1% Low Fat Milk
                             ...                        
255                Hill Country Fare Reduced Fat 2% Milk
256                         Hill Country Fare Whole Milk
257                      Hill Country Fare Fat Free Milk
258                                 H‑E‑B Chocolate Milk
259                                 H‑E‑B Chocolate Milk
Name: type, Length: 260, dtype: object

In [19]:
for index in clean_heb_df.index:
    clean_heb_df['type'][index]=clean_heb_df['type'][index][len(clean_heb_df['brand'][index]):]

<ipython-input-19-18c1788e624b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_heb_df['type'][index]=clean_heb_df['type'][index][len(clean_heb_df['brand'][index]):]


In [20]:
clean_heb_df.groupby(['type']).agg({'type':'count'})

,type
type,
1% Low Fat Milk,2
1% Lowfat Milk,2
100% Lactose Free 2% Reduced Fat Milk,1
100% Lactose Free Calcium Enriched 2% Reduced Fat Milk,1
2% Chocolate Reduced Fat Milk,1
...,...
Vanilla Rice Drink,1
Vanilla Soy Milk,1
Vanilla Walnut Milk,1


In [21]:
clean_heb_df['simple_type']=clean_heb_df['type']

In [33]:
type_list=['almond','chocolate','whole','organic','oat','lactose free','low fat','reduced fat', 'powder','protein','other']
type_list

['almond',
 'chocolate',
 'whole',
 'organic',
 'oat',
 'lactose free',
 'low fat',
 'reduced fat',
 'powder',
 'protein',
 'other']

In [34]:
milk_type_df=pd.DataFrame(columns=type_list)
milk_type_df['name']=clean_heb_df['type']
milk_type_df['id']=clean_heb_df['id']

In [37]:
for type in type_list:
    milk_type_df[type]=clean_heb_df['type'].str.contains(type, case=False)

In [30]:
for index in clean_heb_df.index:
    if clean_heb_df['type'][index].contains('almond', case=False):
        print (clean_heb_df['type'][index].lower())
#        clean_heb_df['type'][index]='whole milk'
        print (clean_heb_df['type'][index])

AttributeError: 'str' object has no attribute 'contains'

In [425]:

for index in clean_heb_df.index:
    if fuzz.partial_ratio(clean_heb_df['type'][index].lower(),'select whole milk')>=76:
        print (clean_heb_df['type'][index].lower())
#        clean_heb_df['type'][index]='whole milk'
        print (clean_heb_df['type'][index])

whole milk
whole milk
 select ingredients mootopia lactose free whole milk
 Select Ingredients MooTopia Lactose Free Whole Milk
 select ingredients whole milk
 Select Ingredients Whole Milk
 select ingredients whole milk
 Select Ingredients Whole Milk
whole milk
whole milk
whole milk
whole milk
 select ingredients whole milk
 Select Ingredients Whole Milk
 select ingredients whole milk
 Select Ingredients Whole Milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk
whole milk


In [1]:
clean_heb_df.groupby(['type']).agg({'id':'count'}).sort_values('id', ascending=False)

NameError: name 'clean_heb_df' is not defined

In [38]:
milk_type_df

,almond,chocolate,whole,organic,oat,lactose free,low fat,reduced fat,powder,protein,other,name,id
0,False,False,True,True,False,False,False,False,False,False,False,Organics Whole Milk,931316
1,False,False,False,True,False,False,False,True,False,False,False,Organics Reduced Fat 2% Milkfat Milk,931317
2,False,False,False,False,False,True,False,True,False,False,False,Select Ingredients MooTopia Lactose Free 2% R...,671076
3,False,False,False,False,False,False,False,False,False,False,False,Select Ingredients Fat Free Milk,314136
4,False,False,False,False,False,False,True,False,False,False,False,Select Ingredients 1% Low Fat Milk,314134
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,False,False,False,False,False,False,False,True,False,False,False,Reduced Fat 2% Milk,314131
256,False,False,True,False,False,False,False,False,False,False,False,Whole Milk,314124
257,False,False,False,False,False,False,False,False,False,False,False,Fat Free Milk,314128
258,False,True,False,False,False,False,False,False,False,False,False,Chocolate Milk,658779
